# Part 2

## Creating New Metaclasses

In [1]:
%less slide5_dynamic_class.py


class MyClass:

    x = 42

    def double(self):
        return self.x * 2


class MyMixin:

    def increase(self):
        return self.x + 2


class MySubClass(MyClass, MyMixin):

    x = 36


def my_subclass_builder(new_x):

    class MyDynamicSubClass(MyClass, MyMixin):

        x = new_x

    return MyDynamicSubClass


In [2]:
%less slide9_better_repr.py

class better_repr_type(type):

    def __init__(cls, name, bases, namespace_dict):

        # cls above is the class under construction

        def __repr__(self):
            attr_repr = ', '.join(
                f'{name}={value!r}'
                for name, value in self.__dict__.items()
            )
            cls = self.__class__
            return f'{cls.__name__}({attr_repr})'

        # Injecting instance method:
        cls.__repr__ = __repr__

        super().__init__(name, bases, namespace_dict)

    # Method for use by the class under construction itself
    def __repr__(cls):
        bases = ', '.join(c.__name__ for c in cls.__bases__)
        return f'{cls.__name__}({bases})'

    x = 25


In [3]:
from slide5_dynamic_class import MyClass, MyMixin

from slide9_better_repr import better_repr_type

def __init__(self, x):
    self.x = x

In [4]:
MySubClassWithRepr = better_repr_type(
    'MySubClassWithRepr',  # name
    (MyClass, MyMixin),  # bases
    {'__init__': __init__},   # attributes / methods
)

In [5]:
MySubClassWithRepr

MySubClassWithRepr(MyClass, MyMixin)

## Using Metaclasses in "Normal" Classes


In [6]:
from slide5_dynamic_class import MyClass

from slide9_better_repr import better_repr_type

class MySubClassWithRepr2(MyClass, metaclass=better_repr_type):
    def __init__(self, x):
        self.x = x


In [7]:
MySubClassWithRepr2

MySubClassWithRepr2(MyClass)

## Class Decorators


In [8]:
%less slide20_meta_alternatives.py

from collections.abc import Mapping


class MetaWaterfowl(type):

    def __init__(cls, cls_name, bases, cls_dict, **kw):
        print('** __init__ in metaclass for', cls_name)
        super().__init__(cls_name, bases, cls_dict, **kw)

    def __repr__(cls):
        cls_name = cls.__name__
        return f"<Class for making {cls_name!r}>"


class Waterfowl(metaclass=MetaWaterfowl):

    registry: Mapping[str, type] = {}

    def __init_subclass__(subclass):
        print('** registering', subclass.__name__)
        subclass.registry[subclass.__name__] = subclass

    def __class_getitem__(cls, name):
        return cls.registry[name]


VALID_ANATIDAE = {
    'Duck',
    'Goose',
    'Drake',
    'Swan',
}

def check_waterfowl(cls):
    print("** verifying:", cls.__name__)
    name = cls.__name__
    if name not in VALID_ANATIDAE:
        raise RuntimeError(
            f"What a quack! {name} is not quacky enough."
        )
    print('** verified:', cls.__name__)
    return cls


In [9]:
from slide20_meta_alternatives import *

@check_waterfowl
class Duck(Waterfowl):

    def quack(self):
        print("Quack!")

Waterfowl['Duck']

** __init__ in metaclass for Waterfowl
** registering Duck
** __init__ in metaclass for Duck
** verifying: Duck
** verified: Duck


<Class for making 'Duck'>

In [10]:
class Cat(Duck):
    def quack(self):
        print("meow!")

** registering Cat
** __init__ in metaclass for Cat


In [11]:
@check_waterfowl
class Dog(Duck):
    def quack(self):
        print("woof, woof!")

** registering Dog
** __init__ in metaclass for Dog
** verifying: Dog


RuntimeError: What a quack! Dog is not quacky enough.

In [12]:
print('End of this notebook')

End of this notebook
